# Getting started
This notebook demonstrates the basic functionality of the HPS API and shows how to start a training session and run evaluations. 

In [ ]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
import pandas as pd
import numpy as np
from hps_api_client.apiproxy import ApiProxy

## Create client
 - The ApiProxy client facilitates access to the api functions and provides classes that wrap the json objects which are exchanged with the server
 - The proxy is preconfigured to point at the api

In [ ]:
client = ApiProxy(uri = "http://leviathan:5400/api/v1/")
# help(ApiProxy)

## List available projects
 - Projects are a means to organize trainig sessions
 - All projects created by any user will be listed

In [ ]:
projects = client.get_projects()
projects

## List hydrosystems
Hydro systems are pre-configured for our customers and cannot be added through the API.
 

In [ ]:
hydro_systems = client.get_hydro_systems()
hydro_system = hydro_systems[-1]
hydro_systems

## Create a new project
For this tutorial we will create a new project for the hydro system Flørli

In [ ]:
project_name = "Demo II"
selected_hydro_system = hydro_system
new_project = client.create_project(project_name, selected_hydro_system)
new_project

## Get forecasts
We are going to pick a forecast to train our agent on. First we list available forecasts.

In [ ]:
forecasts = client.get_forecasts(selected_hydro_system)
forecasts

We will use the most recent forecast 

In [ ]:
selected_forecast = forecasts[0]
selected_forecast

## Get reservoirs for hydro system

In [ ]:
reservoirs = client.get_reservoirs(selected_hydro_system)
reservoirs

## Start training
When starting a training run, a settings object has to be provided. A template for these settings can be retrieved from the API.

In [ ]:
settings = client.get_settings_template(new_project)
print(settings.to_json())

Before starting the run, we will modify some of the settings

In [ ]:
# It is impotant to give the run a description, so we can identfy it later
settings.comment = "demo run"

# We are setting all initial reservoir volums to half capasity
settings.startVolumes = {}
for r in reservoirs:
    settings.startVolumes[r.name] = r.maxVolume / 2

We can now start the trainig session in the context of our new project and using the forcast and settings as specified above

In [ ]:
run = client.run(new_project, selected_forecast, settings)
run

As we can see above, initially the run has neither start- nor endTime specified. As soon as the backend starts the training process, the startTime will be set. When the training process has completed the endTime will also be specified.
We can now call get_runs for our project to the status of our run.

In [ ]:
runs = client.get_runs(new_project)
runs

During training it can be useful to monitor the progress. The show_progress method will display the maximum reward achieved  over time, and can be a good way to identify convergence.

In [ ]:
client.show_progress(run)

At any time during or after training we can retrieve the best solution produced by the agent.

In [ ]:
client.plot_solution(run)

A training session can be terminated before it has run to completion.

In [ ]:
client.terminate_run(run)

## Evaluation
During training we normally evaluate only a subset of the forecast scenarios to determine convergence of the solution. After or during training we can perform more detailed evaluations.

In [ ]:
settings.comment = "demo evaluation"
settings.evaluationEpisodes = 8
evaluation = client.evaluate(run, selected_forecast, settings)
evaluation

As with a trainig session, the evaluation is completed when the endTime is set.

In [ ]:
client.get_evaluations(new_project)

We can plot the results of the evaluation

In [ ]:
client.plot_evaluation(evaluation)

And for further processing, we can retrieve the result data as a pandas DataFrame

In [ ]:
df = client.get_evaluation(evaluation)
df

In [ ]:
df['res2'].plot(figsize=(20,15))